In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import tensorflow as tf

from camels_aus.repository import CamelsAus
from model.tf.hydro import ProductionStorage
from data.tf.camels_dataset import CamelsDataset

2024-08-29 11:28:18.512846: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
window_size = 7
camels_dir = '../data/camels/aus'

In [ ]:
camels_ds = CamelsDataset(data_dir=camels_dir, 
                          station_list=['314213'],
                          # state_outlet='WA', map_zone=50
                          target_vars=['flow_cdf'],
                          window_size=window_size)
train_ds, test_ds = camels_ds.get_datasets(batch_size=256)

In [21]:
for batch in train_ds:
    print(batch['station_id'].shape, batch['timeseries'].shape, batch['static'].shape, batch['target'].shape)
    break

(256, 1) (256, 7, 4) (256, 7) (256, 1)


In [23]:
gr4j_logfile = '/Users/akap5486/Library/CloudStorage/OneDrive-UNSW/Shared/Projects/01_PhD/04_DeepGR4J_QNN/deepgr4j-extremes/results/gr4j/result.csv'
hybrid_ds = HybridDataset(data_dir=camels_dir, gr4j_logfile=gr4j_logfile, 
                        #   station_list=['314213'],
                          state_outlet='WA', map_zone=50)

In [24]:
train_ds, test_ds = hybrid_ds.get_datasets(batch_size=256)

In [26]:
for batch in train_ds:
    print(batch['station_id'].shape, batch['timeseries'].shape, batch['static'].shape, batch['target'].shape)
    break

(256, 1) (256, 7, 8) (256, 7) (256, 1)
